# Bag of Words (Bow)

- Utilizou-se o CountVectorizer do Scikit-learn.
- Ele criou um vocabulário com as 10.000 palavras mais frequentes ( max_features=10000) encontradas em todas as tags (ignorando stop words em inglês).
- Para cada filme, ele gerou um vetor onde cada posição correspondia a uma palavra do vocabulário, e o valor naquela posição era a contagem bruta de quantas vezes aquela palavra aparecia na tag do filme.
- O resultado foi a matriz bow_matrix, onde cada linha é o vetor BoW de um filme.

# TF-IDF

- Utilizou-se o TfidfVectorizer do Scikit-learn.
- Ele também criou um vocabulário com as 10.000 palavras mais frequentes ( max_features=10000, ignorando stop words).
- Para cada filme, ele gerou um vetor onde cada posição correspondia a uma palavra do vocabulário, mas o valor naquela posição não era a contagem bruta, e sim o score TF-IDF daquela palavra naquela tag. Esse score considera tanto a frequência da palavra na tag atual (TF) quanto a raridade da palavra em todas as tags (IDF). Palavras frequentes no filme, mas raras no geral, recebem um score maior.
- O resultado foi a matriz tfidf_matrix, onde cada linha é o vetor TF-IDF de um filme.

## Entendendo o Score TF-IDF

O **TF-IDF** significa **Term Frequency - Inverse Document Frequency** (Frequência do Termo - Frequência Inversa do Documento). É uma medida estatística usada para avaliar a **importância de uma palavra (termo) em um documento específico dentro de uma coleção de documentos (corpus)**. A ideia principal é dar mais peso às palavras que são:

1.  **Frequentes** no documento atual (indicando relevância local).
2.  **Raras** no conjunto geral de documentos (indicando que a palavra é mais discriminativa ou informativa).

O score TF-IDF para uma palavra *t* em um documento *d* é calculado multiplicando dois componentes:

$ TFIDF(t, d) = TF(t, d) \times IDF(t) $

Vamos entender cada componente:

### 1. TF (Term Frequency - Frequência do Termo)

* **O que é:** Mede a frequência com que um termo *t* aparece no documento *d*.
* **Intuição:** Quanto mais vezes uma palavra aparece em um texto, maior a probabilidade de ela ser relevante *para aquele texto específico*.
* **Cálculo:** Existem várias formas de calcular o TF, as mais comuns são:
    * **Contagem Bruta:** Simplesmente o número de vezes que *t* aparece em *d*. (É comum o Scikit-learn usar isso internamente no cálculo do TF-IDF).
    * **Frequência Normalizada:** Contagem bruta dividida pelo número total de palavras no documento *d*. Isso evita que documentos mais longos tenham TFs artificialmente maiores.
    * **Escala Logarítmica:** $ 1 + \log(Contagem Bruta) $ (se a contagem for > 0, senão 0). Isso suaviza o efeito de contagens muito altas.
* **Resultado:** Um valor que representa a importância *local* do termo no documento.

### 2. IDF (Inverse Document Frequency - Frequência Inversa do Documento)

* **O que é:** Mede o quão *informativa* ou *rara* é uma palavra *t* em toda a coleção de documentos. Funciona como um fator de ponderação ou penalidade.
* **Intuição:** Palavras que aparecem em *muitos* documentos (como "o", "a", "é", "filme", "história") são menos informativas para distinguir um documento dos outros e devem receber um peso menor. Palavras que aparecem em *poucos* documentos (como "super-herói", "pandemia", "viagem no tempo") são mais discriminativas e devem receber um peso maior.
* **Cálculo:** A fórmula padrão envolve o número total de documentos no corpus (N) e o número de documentos que contêm o termo *t* ($df_t$ - document frequency):

    $ IDF(t) = \log\left(\frac{N}{df_t}\right) $

    * **Na prática (e no Scikit-learn):** Usa-se frequentemente uma versão suavizada para evitar divisão por zero (se $df_t=0$) e para evitar que o IDF seja zero se uma palavra aparece em todos os documentos:

        $ IDF(t) = \log\left(\frac{N+1}{df_t+1}\right) + 1 $

* **Resultado:** Um valor alto para termos raros e um valor baixo para termos comuns no corpus.

### O Score TF-IDF Final

* Ao multiplicar $ TF(t, d) \times IDF(t) $, obtemos o score TF-IDF.
* **Significado do Score:**
    * **Alto:** A palavra *t* aparece com frequência no documento *d* (TF alto) **E** é relativamente rara no restante do corpus (IDF alto). Essas são as palavras consideradas mais importantes e características daquele documento específico.
    * **Médio:** Pode ocorrer se a palavra for muito frequente no documento mas também comum no corpus (TF alto, IDF baixo), ou se for rara no corpus mas aparecer poucas vezes no documento (TF baixo, IDF alto).
    * **Baixo/Zero:** A palavra *t* é muito comum em todo o corpus (IDF baixo), ou simplesmente não aparece (ou aparece muito pouco) no documento *d* (TF baixo).

**Exemplo Conceitual (Tags de Filmes):**

Imagine as tags:
* Filme A: "Ação herói espacial salva galáxia"
* Filme B: "Ação herói combate robôs"
* Filme C: "Comédia romântica espacial"

* **"Ação", "herói":** TF alto em A e B, mas como aparecem em 2 de 3 documentos, o IDF não será o mais alto. Score TF-IDF será moderado/alto em A e B.
* **"espacial":** TF alto em A e C, IDF não tão alto. Score TF-IDF moderado/alto em A e C.
* **"galáxia":** TF alto só em A, IDF alto (só aparece em 1 de 3). Score TF-IDF **muito alto** em A.
* **"robôs":** TF alto só em B, IDF alto. Score TF-IDF **muito alto** em B.
* **"Comédia", "romântica":** TF alto só em C, IDF alto. Score TF-IDF **muito alto** em C.

**Conclusão:** O TF-IDF gera vetores onde os componentes não são apenas contagens, mas sim pesos que refletem a importância relativa de cada palavra para descrever e distinguir um documento dentro da coleção. Isso geralmente leva a cálculos de similaridade mais significativos do que o BoW.

*(Data da explicação: 29 de abril de 2025)*

# Similaridade de Cossenos

A **Similaridade de Cossenos** é uma métrica usada para medir o quão similares são dois vetores, focando na **direção** para a qual eles apontam, e não em suas magnitudes (ou "tamanhos").

* **Ideia Central:** Calcula o cosseno do ângulo ($ \theta $) entre dois vetores.
    * Se os vetores apontam exatamente na **mesma direção**, o ângulo é 0°, e a similaridade de cossenos é **1**.
    * Se os vetores são **ortogonais** (formam um ângulo de 90°), eles apontam em direções independentes, e a similaridade é **0**.
    * Se os vetores apontam em **direções opostas**, o ângulo é 180°, e a similaridade é **-1**.
* **Uso em Texto:** Quando representamos textos (como as `tags` dos filmes) como vetores (usando BoW, TF-IDF ou SBERT), a similaridade de cossenos nos diz o quão parecidos eles são em termos de **conteúdo ou tema**, independentemente do comprimento do texto original. Textos com palavras-chave ou significados semelhantes terão vetores apontando em direções parecidas, resultando em alta similaridade.
* **Fórmula (Resumida):** É calculada dividindo o produto escalar dos vetores pelo produto de suas magnitudes:
    $ \text{Similaridade}(A, B) = \frac{A \cdot B}{\|A\| \|B\|} $
* **Faixa Comum:** Para vetores de texto que geralmente não têm valores negativos (como os de BoW, TF-IDF, SBERT), a similaridade varia entre **0 (completamente diferentes)** e **1 (idênticos em termos de direção/conteúdo relativo)**.

**Exemplo Simples:**

Imagine um vocabulário minúsculo: "Ação", "Comédia".

* **Filme A:** "Pura Ação" -> Vetor A = `[1, 0]` (Tem "Ação", não tem "Comédia")
* **Filme B:** "Só Comédia" -> Vetor B = `[0, 1]` (Não tem "Ação", tem "Comédia")
* **Filme C:** "Ação e Comédia" -> Vetor C = `[1, 1]` (Tem ambos)

Calculando a similaridade de cossenos:

* **Similaridade(A, B):** Os vetores `[1, 0]` e `[0, 1]` são ortogonais (ângulo 90°). Similaridade = **0**. (Não têm nada em comum no nosso vocabulário).
* **Similaridade(A, C):** Os vetores `[1, 0]` e `[1, 1]` compartilham a dimensão "Ação". O ângulo é menor que 90°. Similaridade > **0** (ex: $\approx 0.707$). (Compartilham um aspecto).
* **Similaridade(A, A):** Comparando `[1, 0]` com ele mesmo. O ângulo é 0°. Similaridade = **1**. (Idênticos).

Em resumo, a similaridade de cossenos foca na proporção dos componentes (palavras/scores/significados) em comum, ignorando o tamanho absoluto (comprimento do texto/magnitude do vetor).

# Embedding

**Representar Significado (Semântica)**: Diferente de vetores simples como os do Bag of Words (que só contam palavras), os embeddings gerados por modelos como o Sentence-BERT são projetados para capturar o significado ou a semântica do texto original. Cada dimensão do vetor embedding (ex: as 768 dimensões do all-mpnet-base-v2) representa alguma característica latente do significado aprendida pelo modelo.

- Exemplo: Filmes com tags que falam sobre "viagem espacial", "alienígenas" e "planetas distantes" terão embeddings que apontam para uma direção similar no espaço vetorial, mesmo que usem palavras ligeiramente diferentes. Filmes sobre "romance medieval" apontarão para uma direção muito diferente.

# O que é UMAP? (Explicação)

**UMAP** significa **Uniform Manifold Approximation and Projection** (Aproximação e Projeção Uniforme de Variedades). É uma técnica moderna e muito popular para **redução de dimensionalidade**.

**Objetivo Principal:**

Seu objetivo é pegar dados de alta dimensão (como nossos embeddings SBERT, que têm centenas de dimensões) e criar uma representação fiel desses dados em um espaço de **dimensão muito menor** (por exemplo, 2, 3, 10 ou 50 dimensões), preservando o máximo possível da **estrutura essencial** dos dados originais. Ele é especialmente bom em capturar tanto a estrutura *local* (quais pontos são vizinhos próximos) quanto, até certo ponto, a estrutura *global* (como os grupos de pontos se relacionam entre si).

**Como Funciona (Intuição):**

Imagine que seus dados de alta dimensão formam uma espécie de "mapa" complexo e dobrado em muitas dimensões (uma "variedade" ou *manifold*). O UMAP tenta fazer o seguinte:

1.  **Mapear Conexões no Espaço Original:** Primeiro, ele constrói uma representação (um tipo de grafo ponderado) de como os pontos estão conectados localmente no espaço original de alta dimensão. Ele olha para os vizinhos mais próximos de cada ponto e modela a probabilidade de eles estarem conectados, considerando a densidade local.
2.  **Encontrar um "Mapa" Similar em Baixa Dimensão:** Em seguida, ele tenta encontrar uma disposição para esses mesmos pontos no espaço de baixa dimensão (ex: 2D ou 10D) de forma que a estrutura de conexões nesse novo espaço seja o mais parecida possível com a estrutura que ele encontrou no espaço original.
3.  **Otimização:** Ele usa um processo de otimização inteligente para ajustar as posições dos pontos no espaço de baixa dimensão até que a correspondência entre as estruturas (alta dimensão vs. baixa dimensão) seja a melhor possível. Pontos que estavam "próximos" ou "conectados" no espaço original devem acabar próximos no espaço de baixa dimensão.

**Principais Parâmetros:**

* `n_neighbors`: Controla quantos vizinhos são considerados para definir a estrutura local. Valores menores focam em detalhes muito locais; valores maiores tentam capturar uma visão mais global. (Default=15).
* `min_dist`: Controla a distância mínima permitida entre pontos no espaço de baixa dimensão. Valores menores criam agrupamentos mais densos e compactos; valores maiores permitem que os pontos se espalhem mais. (Default=0.1).
* `n_components`: O número de dimensões que você deseja no resultado final (ex: 2 para visualização, 5, 10, etc., para usar em outros algoritmos).
* `metric`: A métrica de distância usada para encontrar vizinhos no espaço original. Para embeddings SBERT, `'cosine'` é a escolha apropriada.

**Por Que Usamos UMAP no Nosso Projeto?**

A ideia foi que os embeddings SBERT originais, apesar de ricos semanticamente, poderiam ser muito complexos (alta dimensionalidade, talvez com alguma redundância ou ruído) para que algoritmos como K-Means ou DBSCAN encontrassem facilmente clusters claros e bem definidos (como vimos pelas métricas).

Ao aplicar UMAP *antes* do clustering, buscamos:
1.  **Simplificar os Dados:** Reduzir a complexidade e o ruído, mantendo a estrutura semântica essencial.
2.  **Facilitar o Clustering:** Criar uma representação onde os grupos temáticos pudessem se tornar mais evidentes e separáveis para o K-Means (ou outro algoritmo).
3.  **Melhorar Métricas:** Potencialmente obter resultados mais claros nos métodos de Cotovelo e Silhueta aplicados *após* o UMAP.

Em resumo, UMAP é uma ferramenta para "desenrolar" e simplificar a representação dos seus dados de alta dimensão, tornando a estrutura subjacente mais acessível para análise, visualização ou algoritmos subsequentes como o clustering.

### Explicação: PCA (Principal Component Analysis)

**PCA (Análise de Componentes Principais)** é uma das técnicas mais clássicas e fundamentais para redução de dimensionalidade.

* **Objetivo Principal:** Transformar um conjunto de dados com muitas variáveis (dimensões), possivelmente correlacionadas, em um novo conjunto de variáveis **linearmente não correlacionadas** chamadas **Componentes Principais**. Essas componentes são ordenadas de forma que a primeira captura a maior parte da **variância** dos dados originais, a segunda captura a maior parte da variância restante (sendo ortogonal à primeira), e assim por diante. A redução de dimensionalidade é feita mantendo apenas as primeiras 'k' componentes principais que capturam a maior parte da variância total.
* **Como Funciona (Intuição):** Ele encontra novas "direções" (eixos) no espaço dos dados. A primeira direção é aquela ao longo da qual os dados se "espalham" mais (maior variância). A segunda direção é a que tem a maior variância *restante*, mas que é perpendicular (ortogonal) à primeira, e assim sucessivamente. Ele então projeta os dados originais nesses novos eixos (componentes principais). É uma transformação **linear**.
* **Vantagens:**
    * Matematicamente bem definido, determinístico (sem aleatoriedade).
    * Rápido computacionalmente.
    * Bom para compressão de dados e remoção de multicolinearidade.
    * Os componentes podem ter alguma interpretabilidade em termos de variância explicada.
* **Desvantagens:**
    * **Assume relações lineares** nos dados. Pode não ser eficaz em capturar estruturas complexas e não-lineares (como as que podem existir em embeddings semânticos).
    * Foca em maximizar a variância global, o que **não garante** a preservação da estrutura de clusters ou vizinhanças locais. Dados podem ficar "esmagados" na projeção se a separação não ocorrer nas direções de maior variância.
    * Sensível à escala das variáveis originais (requer padronização/normalização prévia, embora menos crítico para embeddings que já podem ter normas semelhantes).
* **Uso Típico:** Pré-processamento, compressão de dados, redução de ruído (removendo componentes de baixa variância). Menos usado que UMAP/t-SNE especificamente para visualização ou pré-processamento para clustering de dados complexos.

---

### Explicação: t-SNE (t-distributed Stochastic Neighbor Embedding)

**t-SNE** é uma técnica de redução de dimensionalidade muito popular, especialmente para **visualização** de dados de alta dimensão em 2D ou 3D.

* **Objetivo Principal:** Criar uma representação de baixa dimensão onde as **similaridades entre os pontos de dados sejam preservadas**, com foco principal na **estrutura local** (manter vizinhos próximos juntos).
* **Como Funciona (Intuição):**
    1.  **Similaridades no Espaço Original:** Modela a probabilidade de um ponto escolher outro como seu vizinho no espaço de alta dimensão (usando uma distribuição Gaussiana centrada em cada ponto). Pontos próximos têm alta probabilidade mútua.
    2.  **Similaridades no Espaço Reduzido:** Modela probabilidades similares no espaço de baixa dimensão, mas usa uma distribuição com "caudas mais pesadas" (a distribuição t de Student com 1 grau de liberdade). Isso permite que pontos moderadamente distantes no espaço original possam ser modelados mais longe no espaço reduzido, ajudando a separar melhor os clusters e evitar o "amontoamento" de pontos.
    3.  **Otimização:** Ajusta as posições dos pontos no espaço de baixa dimensão para minimizar a diferença (divergência KL) entre as probabilidades do espaço original e do espaço reduzido. O foco é fazer com que os pontos que *eram* vizinhos próximos *continuem* sendo vizinhos próximos.
* **Vantagens:**
    * **Excelente para visualização:** Frequentemente produz mapas 2D/3D visualmente atraentes que revelam claramente agrupamentos (clusters) presentes nos dados.
    * Ótimo em preservar a estrutura local e separar clusters distintos.
* **Desvantagens:**
    * **Não preserva bem distâncias globais:** O tamanho de um cluster ou a distância *entre* clusters diferentes no gráfico t-SNE geralmente **não são significativos**. Ele pode expandir clusters densos e comprimir espaços vazios.
    * **Computacionalmente intensivo:** Geralmente mais lento que PCA ou UMAP, especialmente para muitos pontos.
    * **Estocástico:** O resultado pode variar ligeiramente entre execuções diferentes (a menos que se fixe um `random_state`).
    * **Principalmente para visualização:** Usar as coordenadas t-SNE diretamente como entrada para algoritmos de clustering (como K-Means) pode ser problemático devido à distorção das distâncias globais.
    * Requer ajuste de parâmetros como `perplexity` (relacionado ao número de vizinhos considerados) e `learning_rate`.
* **Uso Típico:** Visualização e exploração de dados de alta dimensão para identificar a presença de clusters ou estruturas locais.

**Comparativo Rápido (PCA vs t-SNE vs UMAP):**

* **PCA:** Linear, foca na variância global, rápido, bom para compressão. Ruim para estruturas não-lineares complexas e preservação local.
* **t-SNE:** Não-linear, foca na estrutura local, ótimo para visualização de clusters. Ruim para preservar estrutura global, mais lento, estocástico.
* **UMAP:** Não-linear, tenta balancear estrutura local e global, geralmente mais rápido que t-SNE, também bom para visualização e frequentemente considerado melhor como pré-processamento para clustering do que t-SNE.

*(Data da explicação: 29 de abril de 2025, João Pessoa)*

### Score de Qualidade (Weighted Rating - WR)

**O Problema:**

Ao olharmos para as notas dos filmes, usar apenas a nota média (`vote_average`) pode ser enganoso. Um filme pode ter uma nota média altíssima (ex: 9.5 de 10), mas baseada em pouquíssimos votos (ex: 10 votos). Por outro lado, um filme com nota 8.5, mas com milhares de votos, provavelmente tem uma avaliação de qualidade mais confiável e representa um consenso maior. Como comparar ou ranquear esses filmes de forma justa?

**O Objetivo do Weighted Rating:**

Criar uma única **pontuação (score)** para cada filme que leve em conta **tanto a sua nota média quanto o número de votos que recebeu**, de forma ponderada. A ideia é dar mais "confiança" à nota média de filmes com muitos votos e ajustar a nota de filmes com poucos votos na direção da média geral do site/dataset.

**A Fórmula (Usada no Projeto):**

A fórmula que implementamos é a seguinte:

$$ WR = \left( \frac{v}{v+m} \right) \times R + \left( \frac{m}{v+m} \right) \times C $$

Onde:

* `WR`: É o Weighted Rating final calculado – o nosso "Score de Qualidade".
* `v`: É o número de votos (`vote_count`) que o filme específico recebeu.
* `R`: É a nota média (`vote_average`) do filme específico.
* `m`: É um parâmetro que define o **número mínimo de votos** necessários para que a nota média do filme seja considerada "confiável" (no nosso caso, definimos como o quantil 90% da contagem de votos - ou seja, um filme precisava ter mais votos que 90% dos outros filmes para ter sua nota R considerada com mais peso). Filmes com menos votos que `m` terão suas notas puxadas para a média geral.
* `C`: É a nota média (`vote_average`) de **todos** os filmes no dataset completo. Funciona como uma linha de base ou "média do site".

**Como Interpretar a Fórmula (Intuição):**

* A fórmula calcula uma **média ponderada** entre a nota do próprio filme (`R`) e a nota média geral (`C`).
* Os pesos ($ \frac{v}{v+m} $ e $ \frac{m}{v+m} $) dependem de quantos votos (`v`) o filme tem em relação ao mínimo `m`.
    * **Se um filme tem MUITOS votos (`v >> m`):** O peso $ \frac{v}{v+m} $ fica próximo de 1, e o peso $ \frac{m}{v+m} $ fica próximo de 0. O `WR` será muito parecido com a nota média original `R` do filme. (O sistema "confia" na nota).
    * **Se um filme tem POUCOS votos (`v << m`):** O peso $ \frac{v}{v+m} $ fica próximo de 0, e o peso $ \frac{m}{v+m} $ fica próximo de 1. O `WR` será muito parecido com a nota média geral `C`. (O sistema "não confia" na nota R e a puxa para a média).
    * **Se `v` é próximo de `m`:** O `WR` será uma mistura das duas notas.

**Uso no Projeto:**

1.  Calculamos esse `WR` para todos os filmes e armazenamos na coluna `'score'` do DataFrame `df`.
2.  Na função `recomendar_por_cluster`, após identificar todos os filmes pertencentes ao mesmo cluster do filme de entrada, usamos a coluna `'score'` para **ordenar** esses filmes.
3.  Dessa forma, as recomendações apresentadas não são apenas filmes do mesmo grupo temático (cluster), mas são os filmes **mais bem avaliados e populares** *dentro* daquele grupo, tornando a lista final mais relevante e útil.

Em resumo, o Score de Qualidade (Weighted Rating) foi uma ferramenta para criar um ranking mais justo e significativo do que usar apenas a nota média bruta, especialmente útil para ordenar as recomendações dentro dos clusters.

*(Data da explicação: 29 de abril de 2025, João Pessoa)*